In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
import scipy.sparse
import time
from sklearn import mixture

In [ ]:
def forward_differences(M,N):
    row = np.arange(0,M*N)
    dat = np.ones(M*N)
    col = np.arange(0,M*N).reshape(M,N)
    col_xp = np.hstack([col[:,1:], col[:,-1:]])
    col_yp = np.vstack([col[1:,:], col[-1:,:]])
    
    FD1 = scipy.sparse.coo_matrix((dat, (row, col_xp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))

    FD2 = scipy.sparse.coo_matrix((dat, (row, col_yp.flatten())), shape=(M*N, M*N))- \
          scipy.sparse.coo_matrix((dat, (row, col.flatten())), shape=(M*N, M*N))
    
    FD = scipy.sparse.vstack([FD1, FD2])
    
    return FD

In [ ]:
def prox_l22(u, f, tau):
    return (u + tau*f)/(1+tau)

def proj_inf_l2(p, tau):
    # size must be (K,N), l2 over K, inf over N
    norm_p = np.sqrt(np.sum(p**2, axis=0, keepdims=True))
    p /= np.maximum(1, norm_p/tau)
    return p

def proj_inf(p, tau):
    return np.clip(p, -tau, tau)

In [ ]:
# accelerated primal dual for ROF
def rof_apd(d, lamb=1.0, maxit=1000, check=100, verbose=0):
    
    M,N = d.shape
    d = d.reshape(M*N)
    u = d.copy()

    # dual variable
    p = np.zeros(M*N*2)

    # make nabla operator
    D = forward_differences(M,N)

    # primal and dual step size
    # tau * sigma * L^2 = 1
    L = np.sqrt(8)
    tau = 1/L    
    sigma = 1/tau/L**2
    theta = 1.0
    
    t0 = time.time()
    G = []
    for it in range(0,maxit):

        # remeber old
        u_ = u.copy()

        # primal update
        u -= tau*(D.T@p)
    
        # proximal maps
        u = prox_l22(u,d,tau)
            
        # overrelaxation
        u_ = u + theta*(u-u_)

        theta = 1/np.sqrt(1+tau)
        sigma = sigma/theta;
        tau = tau*theta;
        
        # dual update
        p += sigma*(D@u_)

        # projection
        p = p.reshape(2,M*N)
        p = proj_inf_l2(p, lamb)
        p = p.reshape(2*M*N)
        
        TV1 = lamb*np.sum(np.sqrt(np.sum(((D@u).reshape(2,M*N))**2, axis=0)))
        TV2 = np.sum((D@u)*p)
        gap = TV1 - TV2 + 0.5*np.sum((d-D.T@p-u)**2)
        
        G.append(gap)
        
        if verbose > 0:
            if it%check == check-1:
                

                print("iter = ", it,
                      ", tau = ", "{:.3f}".format(tau),
                      ", sigma = ", "{:.3f}".format(sigma),
                      ", time = ", "{:.3f}".format(time.time()-t0),
                      ", gap = ", "{:.6f}".format(gap),
                      end="\n")
                
    return u.reshape(M,N), p.reshape(2,M,N), np.array(G)

In [ ]:
# Load image
g = ski.io.imread("romy.jpg")/255.0
M,N,_ = g.shape

#initial mask
mask = np.zeros((M,N))
mask[100:-100, 150:-150]=1

plt.figure()
gg = g.copy()
gg[:,:,0]*= mask
gg[:,:,1]*= mask
gg[:,:,2]*= mask
plt.imshow(gg)

In [ ]:
# number of mixture components
K = 5
lamb_tv = 30.0

# initialize GMMs for foreground and background
gmm_fg = mixture.GaussianMixture(n_components=K, verbose=2)
gmm_bg = mixture.GaussianMixture(n_components=K, verbose=2)

pixels = g.reshape(-1,3)
fig = plt.figure(figsize=(10,5))
for it in range(3):

    # compute GMMs with current mask
    mask = mask.flatten()
    gmm_fg.fit(pixels[mask==1,:])
    gmm_bg.fit(pixels[mask==0,:])
    
    # compute nagtive log ratio
    weight = gmm_fg.score_samples(pixels) - gmm_bg.score_samples(pixels)
    
    # Solve the ROF model using accelerated primal-dual
    u_tv, p_tv, _ = rof_apd(weight.reshape(M,N), maxit=400,lamb=lamb_tv, verbose=1)
    
    # compute segmentation mask
    mask = np.double(u_tv >= 0.0)
    
    gg = g.copy()
    gg[:,:,0]*= mask
    gg[:,:,1]*= mask
    gg[:,:,2]*= mask
    plt.imshow(gg, cmap="gray")
    
    fig.canvas.draw()
    time.sleep(0.00001)